# 使用BERT+LSTM+CRF 实现NER任务

In [3]:
# pip install pytorch-crf seqeval

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

import torch.nn as nn
from torchcrf import CRF

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


# 加载数据

In [5]:

def load_data():
    file_train = '../dataset/msra_ner/train/part.txt'
    df = pd.read_csv(file_train, sep='\t', nrows=1000, header=None)
    df.columns = ['text', 'labels']
    df['text'] = df['text'].apply(lambda x: x.replace('', ' '))
    df['labels'] = df['labels'].apply(lambda x: x.replace('', ' '))

    return df

In [6]:
df = load_data()

In [7]:

def make_label_id_dict(df):
    labels = [x.split() for x in df.labels.values.tolist()]
    # 标签去重
    unique_labels = set()
    for lb in labels:
        [unique_labels.add(i) for i in lb]
        
    # print(unique_labels)
        
    label2id = {v: k for k, v in enumerate(unique_labels)}
    id2label = {k: v for k, v in enumerate(unique_labels)}
    
    return label2id, id2label, unique_labels, labels

label2id, id2label, unique_labels, labels = make_label_id_dict(df)

print(label2id)
print('-' * 20)
print(id2label)

{'B-ORG': 0, 'B-LOC': 1, 'B-PER': 2, 'I-PER': 3, 'I-LOC': 4, 'I-ORG': 5, 'O': 6}
--------------------
{0: 'B-ORG', 1: 'B-LOC', 2: 'B-PER', 3: 'I-PER', 4: 'I-LOC', 5: 'I-ORG', 6: 'O'}


# 构建DataLoader

In [8]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 5

model_path = "../../../models/bert-base-chinese/"
tokenizer = BertTokenizer.from_pretrained(model_path) 

BERT做NER 一个棘手部分是 BERT 依赖于 wordpiece tokenization，而不是 word tokenization。比如：Washington的标签为 "b-gpe",分词之后得到， "Wash", "##ing", "##ton","b-gpe", "b-gpe", "b-gpe"

In [9]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """    
    Word piece tokenization使得很难将词标签与单个subword进行匹配。
    这个函数每次次对每个单词进行一个分词，这样方便为每个subword保留正确的标签。 
    当然，它的处理时间有点慢，但它会帮助我们的模型达到更高的精度。
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split()):

        # 逐字分词
        tokenized_word = tokenizer.tokenize(word) # id
        n_subwords = len(tokenized_word) # 1

        # 将单个字分词结果追加到句子分词列表
        tokenized_sentence.extend(tokenized_word)

        # 标签同样添加n个subword，与原始word标签一致
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [10]:
example = df.iloc[0]
print(example)
tokenize_and_preserve_labels(example['text'], example['labels'], tokenizer)

text                    海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。
labels    O O O O O O O B-LOC I-LOC O B-LOC I-LOC O O O ...
Name: 0, dtype: object


(['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'O',
  'B-LOC',
  'I-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'])

In [11]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # 步骤 1: 对每个句子分词
        sentence = self.data.text[index]  
        word_labels = self.data.labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # 步骤 2: 添加特殊token并添加对应的标签
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # 给[CLS] token添加O标签
        labels.insert(-1, "O") # 给[SEP] token添加O标签

        # 步骤 3: 截断/填充
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # 截断
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # 填充
          tokenized_sentence = tokenized_sentence + ['[PAD]' for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # 步骤 4: 构建attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # 步骤 5: 将分词结果转为词表的id表示
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [12]:
from sklearn.model_selection import train_test_split
# train_dataset,test_dataset=train_test_split(data,test_size=0.2,random_state=42)

train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1000, 2)
TRAIN Dataset: (800, 2)
TEST Dataset: (200, 2)


In [13]:
training_set[5]

{'ids': tensor([ 101, 1377, 3221, 8024, 2357, 5812, 2209, 4638,  976, 3791, 6901, 1168,
          671,  763, 2768, 1447, 1744, 3124, 2424, 7566, 2193, 1469, 3616, 3828,
         6379,  833, 6379, 7270, 4638, 2900, 6569, 8024, 2548, 1744, 2600, 4415,
         4906, 2209, 1469,  800, 4638, 6568, 7270, 7794, 3419, 2209,  738, 1728,
         3634,  679, 4721,  511,  102,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# 定义网络

- 模型结构：BertForTokenClassification- 
预训练权重： "bert-base-uncased"

In [15]:
from torchcrf import CRF

class BertCrfNerModel(nn.Module):
    def __init__(self, num_labels):
        super(BertCrfNerModel, self).__init__()
        
        self.bert = BertForTokenClassification.from_pretrained(model_path, num_labels=num_labels, return_dict=True)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=None)
        logits = outputs.logits

        mask = attention_mask.byte()  # 转换为布尔型张量
        loss = -self.crf.forward(logits, labels, mask, reduction='mean')
        return loss

    def predict(self, x, mask=None):
        """
        预测函数
        x：Bert模型中的input ids
        """
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=None)
        emissions = outputs.logits
        preds = self.crf.decode(emissions, mask)
        # preds = [seq + [-1]*(mask.size(1)-len(seq)) for seq in preds]
        return preds
        

In [16]:

num_labels=len(label2id)

model = BertCrfNerModel(num_labels)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ../../../models/bert-base-chinese/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertCrfNerModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bi

# 训练模型

In [17]:

torch.Size([1, 91])
# unsqueeze(0): 在最前面位置添加1个维度：[100] -> [1, 100]
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0) # 真实标签
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)

print(ids.shape, mask.shape, targets.shape)

outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs
initial_loss.item()

torch.Size([1, 100]) torch.Size([1, 100]) torch.Size([1, 100])


D:\Python\envs\py310\lib\site-packages\torchcrf\__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:519.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


74.90795135498047

模型输出logits大小为 (batch_size, sequence_length, num_labels):

设置优化器Adam

In [18]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# 训练函数
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # 将model设置为train模式
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long) #(4, 100)
        mask = batch['mask'].to(device, dtype = torch.long) #(4, 100)
        targets = batch['targets'].to(device, dtype = torch.long)#(4, 100)
        
        loss = model(input_ids=ids, attention_mask=mask, labels=targets)
        
        tr_loss += loss.item()
        nb_tr_steps += 1
        if idx % 50==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 50 training steps: {loss_step}")
           
    
        # 梯度剪切
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # loss反向求导
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")

In [19]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 50 training steps: 72.27678680419922
Training loss per 50 training steps: 34.38899094450708
Training loss per 50 training steps: 24.47512615317165
Training loss per 50 training steps: 19.7385882320783
Training loss epoch: 16.496800837516783


# 评估模型
验证集评估

In [19]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            # loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=targets)
            y_hat = model.predict(x,mask)
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            eval_logits = outputs[0],outputs[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # 计算准确率
            flattened_targets = targets.view(-1) # 大小 (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # 大小 (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # 大小 (batch_size * seq_len,)
            
            active_accuracy = mask.view(-1) == 1 # 大小 (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.11738655716180801
Validation Loss: 0.06324117779149674
Validation Accuracy: 0.9613891533189418


In [20]:
tmp=[]
for tags in df['labels']:
    tmp.extend(tags.split())
    
pd.Series(tmp).value_counts()

O        41790
I-ORG     1898
I-LOC     1065
I-PER      857
B-LOC      782
B-ORG      481
B-PER      407
Name: count, dtype: int64

In [25]:
print(labels[0:20])
print(len(labels))
print('-' * 40)
print(predictions[0:20])
print(len(predictions))

['O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG']
9373
----------------------------------------
['O', 'O', 'B-LOC', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'I-ORG', 'I-ORG']
9373


In [27]:
from seqeval.metrics import classification_report
print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         LOC       0.54      0.64      0.59       153
         ORG       0.27      0.39      0.32       105
         PER       0.80      0.88      0.84        69

   micro avg       0.49      0.61      0.54       327
   macro avg       0.54      0.64      0.58       327
weighted avg       0.51      0.61      0.55       327



> 结论分类的准召比较差

# 预测

In [29]:
sentence = "侯海伦在石家庄御芝林公司担任算法工程师。"

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# 加载到gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# 输入到模型
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # 大小 (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # 大小 (batch_size*seq_len,) 

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# 拼接文本
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

侯 海 伦 在 石 家 庄 御 芝 林 公 司 担 任 算 法 工 程 师 。
['B-PER', 'I-PER', 'I-PER', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


# 保存模型

In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# 保存tokenizer
tokenizer.save_vocabulary(directory)
# 保存权重和配置文件
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')